In [3]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import gdown
import zipfile
from tensorflow.keras.utils import Sequence


print(tf.__version__)

2.17.0


In [ ]:
file_id = "1oQ6Vy_HqZlVHnkFspgxMn0IcE__D8Kmh"
zip_filename = "ocular-disease-recognition.zip"
extract_path = "./ocular-disease-recognition"

# Check if the file already exists
if not os.path.exists(zip_filename):
    print(f"Downloading {zip_filename}...")
    gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_filename, quiet=False)
else:
    print(f"{zip_filename} already exists. Skipping download.")

In [ ]:
# Check if already extracted
if not os.path.exists(extract_path):
    os.makedirs(extract_path, exist_ok=True)
    print(f"Extracting {zip_filename}...")

    with zipfile.ZipFile(zip_filename, "r") as zip_ref:
        zip_ref.extractall(extract_path)

    print(f"Extraction complete! Files extracted to: {extract_path}")
else:
    print(f"Extraction skipped: {extract_path} already exists.")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import cv2
import numpy as np
import pandas as pd

# Load the dataset (Update the path if necessary)
dataset_path = "processed_ocular_disease.csv"
df = pd.read_csv(dataset_path)

# **Split into Train, Validation, and Test Sets**
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)  # 70% Train, 30% Temp
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)  # 15% Val, 15% Test

In [20]:
from tensorflow.keras.utils import Sequence
import os
import cv2
import numpy as np

class OcularDatasetGenerator(Sequence):
    def __init__(self, df, batch_size=32, img_size=(128, 128), shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.indices = np.arange(len(df))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))  # Number of batches per epoch
    
    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch = self.df.iloc[batch_indices]
        X, y = self.__data_generation(batch)
        return np.array(X), np.array(y)  # Return batch images and labels
    
    def __data_generation(self, batch):
        X_batch = []
        y_batch = []
        for _, row in batch.iterrows():
            left_image_path = os.path.join('ocular-disease-recognition/preprocessed_images', row['Left-Fundus'])
            right_image_path = os.path.join('ocular-disease-recognition/preprocessed_images', row['Right-Fundus'])
            
            left_image = self.load_image(left_image_path)
            right_image = self.load_image(right_image_path)

            if left_image is None or right_image is None:
                continue

            combined_image = np.stack((left_image, right_image), axis=-1)  # Shape: (128, 128, 2)
            X_batch.append(combined_image)

            # Fix: Convert labels to integer type
            y_batch.append(int(row['labels']))  

        return np.array(X_batch, dtype=np.float32), np.array(y_batch, dtype=np.int32)


    
    def load_image(self, image_path):
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            return None
        image = cv2.resize(image, self.img_size) / 255.0  # Normalize
        return image

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

# Create the generator
batch_size = 32
train_generator = OcularDatasetGenerator(df, batch_size=batch_size)


In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import numpy as np


# **Create Data Generators**
batch_size = 32
train_generator = OcularDatasetGenerator(train_df, batch_size=batch_size)
val_generator = OcularDatasetGenerator(val_df, batch_size=batch_size)
test_generator = OcularDatasetGenerator(test_df, batch_size=batch_size, shuffle=False)  # No shuffle for testing

# **Get Number of Classes**
num_classes = len(np.unique(df['labels']))
print(f"Number of Classes: {num_classes}")

# **Define CNN Model**
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 2)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Using computed num_classes
])

# **Compile the model**
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# **Define Early Stopping Callback**
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',    # Stop if validation loss stops improving
    patience=5,            # Wait for 5 epochs before stopping
    restore_best_weights=True  # Restore best model weights
)

# **Train the Model with Early Stopping**
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,  # Validation generator for early stopping
    callbacks=[early_stopping]
)

Number of Classes: 8
Epoch 1/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 22s 145ms/step - accuracy: 0.4238 - loss: 1.6255 - val_accuracy: 0.4302 - val_loss: 1.5690
Epoch 2/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 20s 141ms/step - accuracy: 0.4649 - loss: 1.5134 - val_accuracy: 0.4319 - val_loss: 1.5002
Epoch 3/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 20s 141ms/step - accuracy: 0.4562 - loss: 1.4712 - val_accuracy: 0.4664 - val_loss: 1.4786
Epoch 4/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 20s 140ms/step - accuracy: 0.4763 - loss: 1.3895 - val_accuracy: 0.4625 - val_loss: 1.4522
Epoch 5/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 20s 139ms/step - accuracy: 0.4779 - loss: 1.3560 - val_accuracy: 0.4710 - val_loss: 1.3623
Epoch 6/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 20s 139ms/step - accuracy: 0.5083 - loss: 1.2800 - val_accuracy: 0.4954 - val_loss: 1.3640
Epoch 7/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 20s 138ms/step - accuracy: 0.5084 - loss: 1.2488 - val_accuracy: 0.4798 - val_loss: 1.3384
Epoch 8/50
139/139 ━━━━━━━━━━━━━━━━━━━━ 20s 140ms/step - accur

In [26]:
# **Evaluate the Model on Test Set**
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.4794 - loss: 1.2924
Test Accuracy: 0.4793
